<a href="https://colab.research.google.com/github/ridhanya99/Data-Visualization/blob/main/OnlineActivity2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import folium
import operator

#Data and its Description
https://openflights.org/data.html


#1.	Show all the Indian states  with encoded colour and mark all the airports

In [2]:
#Airport data
airport_col = ['ID', 'Name', 'City', 'Country','IATA', 'ICAO', 'Lat', 'Long', 'Alt', 'Timezone', 'DST', 'Tz database time zone', 'type', 'source']
airport_df = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat",names = airport_col, index_col = 0)

airport_ind = airport_df[(airport_df.Country == "India")]
airport_ind.head()


,Name,City,Country,IATA,ICAO,Lat,Long,Alt,Timezone,DST,Tz database time zone,type,source
ID,,,,,,,,,,,,,
2994,Sardar Vallabhbhai Patel International Airport,Ahmedabad,India,AMD,VAAH,23.077200,72.634697,189,5.5,N,Asia/Calcutta,airport,OurAirports
2995,Akola Airport,Akola,India,AKD,VAAK,20.698999,77.058601,999,5.5,N,Asia/Calcutta,airport,OurAirports
2996,Aurangabad Airport,Aurangabad,India,IXU,VAAU,19.862700,75.398102,1911,5.5,N,Asia/Calcutta,airport,OurAirports
2997,Chhatrapati Shivaji International Airport,Mumbai,India,BOM,VABB,19.088699,72.867897,39,5.5,N,Asia/Calcutta,airport,OurAirports
2998,Bilaspur Airport,Bilaspur,India,PAB,VABI,21.988400,82.111000,899,5.5,N,Asia/Calcutta,airport,OurAirports


In [3]:
india_map =folium.Map(location=[20.5937, 78.9629 ],zoom_start=5, tiles='Stamen Terrain')

for name, row in airport_ind.iterrows():
  folium.Marker(location=[row["Lat"], row["Long"]], popup=row["Name"]).add_to(india_map)

india_map

#2.	Mark the first five countries with highest number of airports(world) – add markers

#3.	Based on the number of airports classify the countries – use colour saturation

In [4]:
count_of_airport_countries = airport_df.groupby(airport_df["Country"])["Name"].count().to_dict()

In [5]:
count_of_airport_countries_sorted = sorted(count_of_airport_countries.items(),key=operator.itemgetter(1),reverse=True)
print(count_of_airport_countries_sorted[0:10])

[('United States', 1512), ('Canada', 430), ('Australia', 334), ('Brazil', 264), ('Russia', 264), ('Germany', 249), ('China', 241), ('France', 217), ('United Kingdom', 167), ('India', 148)]


In [6]:
df_Airport = pd.DataFrame.from_dict(count_of_airport_countries_sorted)
df_Airport.columns=["Country","Airport"]
print(df_Airport.head(5))

         Country  Airport
0  United States     1512
1         Canada      430
2      Australia      334
3         Brazil      264
4         Russia      264


In [7]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

def geolocate(country):
    try:
        loc = geolocator.geocode(country)
        return (loc.latitude, loc.longitude)
    except:
        return np.nan


/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [8]:
top_countries = df_Airport[0:5]
Lat =[]
Long=[]
for index, row in top_countries.iterrows():
    lat,lon = geolocate(row['Country'])
    Lat.append(lat)
    Long.append(lon)
    #print(lat,lon)

top_countries["Lat"] = Lat
top_countries["Long"]=Long
print(top_countries)

         Country  Airport        Lat        Long
0  United States     1512  39.783730 -100.445882
1         Canada      430  61.066692 -107.991707
2      Australia      334 -24.776109  134.755000
3         Brazil      264 -10.333333  -53.200000
4         Russia      264  64.686314   97.745306


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [10]:
world_map = folium.Map(location=[30, 0], zoom_start=2)

folium.Choropleth(
    geo_data=country_shapes,
    name='Choropleth Airport data',
    data=df_Airport,
    columns=['Country', 'Airport'],
    key_on='feature.properties.name',
    #key_on ='feature.id',
    fill_color='YlGn',
    nan_fill_color='white'
).add_to(world_map)

for lat, lon, name,count in   zip(top_countries['Lat'],top_countries['Long'],top_countries['Country'],top_countries['Airport']):
    #folium.Marker(location=[lat, lon],popup=name+"--"+str(count)).add_to(world_map)
    folium.CircleMarker(location = [lat, lon], radius=5, popup=name+" -"+str(count), fill =True).add_to(world_map)

world_map

#4.	Show the distribution of airports – use colour encoding

In [11]:
df_Airport1 = pd.DataFrame.from_dict(count_of_airport_countries.items())

df_Airport1.columns=["Country","Airport"]
print(df_Airport1.head(5))


          Country  Airport
0     Afghanistan       22
1         Albania        5
2         Algeria       44
3  American Samoa        2
4          Angola       25


In [12]:
Lat =[]
Long=[]
for index, row in df_Airport1.iterrows():
    lat,lon = geolocate(row['Country'])
    Lat.append(lat)
    Long.append(lon)
    #print(lat,lon)

df_Airport1["Lat"] = Lat
df_Airport1["Long"]=Long
print(df_Airport1)

            Country  Airport        Lat        Long
0       Afghanistan       22  33.768006   66.238514
1           Albania        5  41.000028   19.999962
2           Algeria       44  28.000027    2.999983
3    American Samoa        2 -14.289304 -170.692511
4            Angola       25 -11.877577   17.569124
..              ...      ...        ...         ...
232       West Bank        1  53.712756   -1.055909
233  Western Sahara        3  24.179732  -13.766785
234           Yemen       11  16.347124   47.891527
235          Zambia       13 -14.518624   27.559916
236        Zimbabwe       16 -18.455496   29.746841

[237 rows x 4 columns]


In [13]:
#---->>To save Country and its latitude and longitude data for easy access in future

from google.colab import drive
drive.mount('drive')
df_Airport1.to_csv('Country_Lat_long.csv')
!cp Country_Lat_long.csv "drive/My Drive/Colab Notebooks/Data Visualization"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [14]:
#bubble map to show the distribution

world_map = folium.Map(location=[30, 0], zoom_start=2)

for lat, lon, name,count in   zip(df_Airport1['Lat'],df_Airport1['Long'],df_Airport1['Country'],df_Airport1['Airport']):
  #folium.CircleMarker(location = [lat, lon], radius=5, popup=name+" -"+str(count), fill =True).add_to(world_map)
  folium.Circle(location= [lat, lon],
      popup=name+" -"+str(count),
      radius=count*1500,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(world_map)

world_map


#5.	Focus on India region to show the airports density


In [23]:
from folium.plugins import HeatMap

india_map = folium.Map(location=[20.5937, 78.9629 ],zoom_start=5, tiles='Stamen Toner')

for name, row in airport_ind.iterrows():
  #folium.Marker(location=[row["Lat"], row["Long"]], popup=row["Name"]).add_to(india_map)
  folium.CircleMarker(location = [row["Lat"], row["Long"]], radius=0.5, popup = row["Name"], fill =True).add_to(india_map)

density_ind = airport_ind[['Lat', 'Long']].values
HeatMap(data = density_ind, radius=19, max_zoom=20).add_to(india_map)

india_map

#6.	Project a heat map on the map layer to  show the airports densities based on time zones.

In [21]:

world_map = folium.Map(location=[30, 0], zoom_start=2)

df1 = (airport_df.groupby([airport_df["Tz database time zone"].fillna('tmp')])
        .agg({'Name': 'count','Lat':'mean','Long':'mean'})
        .reset_index())#.values.tolist())
df1 = df1.rename(columns = {'Name':'Count'},inplace=False)
#print(df1.head(5))

for lat, lon,count,t_zone in   zip(df1['Lat'],df1['Long'],df1['Count'],df1['Tz database time zone']):
    #folium.Marker(location=[lat, lon],popup=t_zone+" -"+str(count)).add_to(world_map)
    folium.CircleMarker(location = [lat, lon], radius=0.5, popup=t_zone+" -"+str(count), fill =True).add_to(world_map)

density = df1[['Lat', 'Long']].values
HeatMap(data = density, radius=15, max_zoom=20).add_to(world_map)

world_map